<a href="https://colab.research.google.com/github/ratmcu/colaboratory_import/blob/master/nb_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import re
import os
import pandas as pd
nltk.download('punkt')
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
import regex

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [0]:
class PageContents():
    '''requests and gathers a Wikipedia page'''
    def __init__(self, url):
        self.quote_page = url
        try:
            self.page = urllib.request.urlopen(self.quote_page)
            self.soup = BeautifulSoup(self.page, 'html.parser')
        except:
            self.table = False
        try:
            self.side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
            self.table_entry_list = self.side_pane.find('tbody').find_all('tr')
            self.table = True
        except:
            self.table = False
    #       print(self.table_entry_list)
    #       self.table_entry_list = self.soup.find('tbody').find_all('tr')

In [0]:
class InfoCard():
    '''accepts a beautiful soup html table from a wikipedia page and scrapes the table to a dictionary'''
    def __init__(self, page_content):
        self.info_table = {}
        if not page_content.table:
            raise RuntimeError('page content is incomplete') from error
        for table_entry in page_content.table_entry_list:
            try:
                row_left = table_entry.find('th', attrs={'scope': 'row'})
                row_right = row_left.next_sibling
                self.info_table[row_left.text] = self.get_text_parts(self.get_kids(row_right))
            except:
                pass
        self.info_table = self.filter_info_scrapes(self.info_table)
        print('info card is scraped successfully')
        
    def get_kids(self, html_mother):
        kid_list = []
        try:
            kids = html_mother.children
            for kid in kids:
                kid_list.append(self.get_kids(kid))
        except:
            return html_mother
        return kid_list 

    def get_text_parts(self, text_lists):
        text_parts = []
        if type(text_lists) == list:
            for element in text_lists:
                text_parts.extend(self.get_text_parts(element))
        else:
            text_parts.append(text_lists)
        return(text_parts)

    def filter_info_scrapes(self, scape_dict):
        mask_dict = {}
        for key, val in scape_dict.items():
            mask_dict[key] = []
            for i, element in enumerate(scape_dict[key]):
                fnd = [c in element for c in u'()[]\n\xa0']
                if not True in fnd:
                    mask_dict[key].append(element)  
        return mask_dict  

In [0]:
# can we use snorkle to get this entity scraping done??? 
# if so it'd be better to have the lable functions in a ready way to generally working on all the pages!!
# 
class PrivateEntities():
    '''given dictionary of a side bar is processed for private entities of interest'''
    def __init__(self, info_card):
        self.info_dict = info_card.info_table
        self._get_entity_dict()
        self._extract_entities()
        
    def _extract_entities(self):
        # TREAT everyone the same? or call a seperate function per each entity?
        # leave the not found entities as empty lists so we can use the html scraper to fill them
        # lets treat everything generally, ultimate filtering happens at the comparison stage(if theres one)
        for entity_key in self.entity_dict.keys():
            self._pick_entity(self.entity_dict[entity_key], entity_key)
            print(self.entity_dict[entity_key])
    
    def _pick_entity(self, entity_list, entity_key):
        nlp = spacy.load("en_core_web_sm")
#         print(entity_list)
        # Find named entities, phrases and concepts
        entity_list.append([])
#         print(entity_list[0][0])
        
        if entity_key == 'BIRTH_PLACE':
            mask_str = ''
            for scrape in self.info_dict[entity_list[0][0]]:
                doc = nlp(str(scrape))
                if doc.ents and doc.ents[0].label_ == 'DATE':
                    mask_str = mask_str + 'd'
                elif str(scrape) == ', ':
                    mask_str = mask_str + 's'
                elif self._is_name(str(scrape)):
                    mask_str = mask_str + 't'
                else:
                    mask_str = mask_str + 'u'
            print(mask_str)
            indices = self._get_target_indices([r'[s][t]',r'[d][t]'], mask_str)  
            for index in indices:
                entity_list[1].append(self.info_dict[entity_list[0][0]][index])
        else:    
            for scrape in self.info_dict[entity_list[0][0]]:
                doc = nlp(str(scrape))
    #             print(scrape)
                if doc.ents and doc.ents[0].label_ == entity_list[0][1]:
                    entity_list[1].append(doc.ents[0].text)
    #                 print(doc.ents[0].text)
                elif not self._entity_noise(entity_key, str(scrape)):
                    entity_list[1].append(str(scrape))
                else:
                    pass
                
    def _get_target_indices(self, pattern_list, mask_str):
        '''TODO: if the target is inside the pattern'''
        indices = []
        for pattern in pattern_list:
            m_iter = re.finditer(pattern, mask_str)
            if m_iter:
                for m in m_iter: 
                    indices.append(m.start()+1)
        return indices        
        
    def _entity_noise(self, entity_key, text):
        try:
            return getattr(self, '_filter_{0}'.format(entity_key))(text)
        except:
            print('filter function for {0} not found'.format(entity_key))
            return True
    
    def _get_entity_dict(self):
        self.entity_dict = {}
        if 'Born' in self.info_dict.keys():
            self.entity_dict['NAME'] = [['Born', 'PERSON']]
            self.entity_dict['BIRTH_DATE'] = [['Born', 'DATE']]
            self.entity_dict['BIRTH_PLACE'] = [['Born', 'GPE']]
        if 'Children' in self.info_dict.keys():
            self.entity_dict['CHILDREN'] = [['Children', 'PERSON']]
        if 'Spouse(s)' in self.info_dict.keys():
            self.entity_dict['SPOUSES'] = [['Spouse(s)', 'PERSON']]
        if 'Parents' in self.info_dict.keys():
            self.entity_dict['PARENTS'] = [['Parents', 'PERSON']]
        if 'Education' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] = [['Education', 'ORG']]
    
    def _filter_NAME(self, text):
        '''cannot simply filter noise due to many possible candidates'''
        return True
    def _filter_BIRTH_DATE(self, text):
        return True
    def _filter_BIRTH_PLACE(self, text):
        return True
    def _filter_CHILDREN(self, text):
#         return False # let's assume we don't find any noisy text under info boxes children till we do an analysis
        return not self._is_name(text)
    def _filter_SPOUSES(self, text):
        return not self._is_name(text)
    def _filter_PARENTS(self, text):
        return not self._is_name(text)
    def _filter_EDUCATION(self, text):
        return True
    def _is_name(self, text):
        p = regex.compile(r"\p{Lu}") # To support (currently) 1702 uppercase letters
#         p = regex.compile(r"[[:upper:]]") # To support (currently) 1822 uppercase letters
        if p.match(text):
            return True
        else:
            return False
  

In [6]:
#experiment_code
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
pg = PageContents('https://en.wikipedia.org/wiki/Joachim_Gauck')


info_card = InfoCard(pg)
# print(type(str(info_card.info_table['Born'][0])))
pe = PrivateEntities(info_card)
# pe.entity_dict

info card is scraped successfully
[['Born', 'PERSON'], []]
[['Born', 'DATE'], ['1940-01-24', '24 January 1940']]
ddtstst
[['Born', 'GPE'], ['Mecklenburg', 'Nazi Germany', 'Rostock']]
[['Children', 'PERSON'], []]
[['Spouse(s)', 'PERSON'], ['Gerhild Radtke  ']]
